In [ ]:
import pandas as pd
# Ajuste das configurações de exibição
pd.set_option('display.float_format', lambda x: '%.2f' % x)
########################################## LENDO A BASE OPERACIONAL PRONTA PARA TRABALHAR
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO\BASE_OPERACIONAL"
base_operacional_hugo = pd.read_parquet(caminho+"/BASE_OPERACIONAL_TELECOB_HUGO.parquet")

# base_operacional_hugo = base_operacional_hugo[['BASE','TELEFONE_HIGIENIZADO','TELEFONE','ATRASO','DEVEDOR','CONTRATO']]

base_operacional_hugo['ATRASO'] = base_operacional_hugo['ATRASO'].astype('int16')
# Criando coluna Faixa de atraso com suas respectivas faixas
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
base_operacional_hugo.loc[base_operacional_hugo.ATRASO.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

In [ ]:
base_operacional_hugo.query("CONTRATO == '002001019034'").drop_duplicates(['CONTRATO','TELEFONE','TELEFONE_HIGIENIZADO'])

In [ ]:
base_1 = base_operacional_hugo[['BASE','TELEFONE_HIGIENIZADO','DEVEDOR','ATRASO','Faixa_de_Atraso','CONTRATO']].rename(columns={'TELEFONE_HIGIENIZADO':'TELEFONE'})
base_2 = base_operacional_hugo[['BASE','TELEFONE','DEVEDOR','ATRASO','Faixa_de_Atraso','CONTRATO']]
bases_teles_tels = pd.concat([base_1,base_2])
bases_teles_tels = bases_teles_tels.drop_duplicates(['BASE','TELEFONE'])
bases_teles_tels = bases_teles_tels.query("BASE != 'Claro MÃ³vel'")
bases_teles_tels = bases_teles_tels.query('TELEFONE.notnull()')


In [ ]:
### Carregando a base de telefones BLACKLIST
blacklist = pd.read_parquet(r"R:\TI\TELEFONIA/BLACKLIST_NPTIM.parquet")
blacklist = blacklist.drop_duplicates(['dddtelefone']).reset_index(drop=True)
blacklist = blacklist.assign(BLACKLIST=1)
blacklist['dddtelefone'] = blacklist['dddtelefone'].astype('str')

In [ ]:
bases_teles_tels = pd.merge(bases_teles_tels, blacklist, right_on=['dddtelefone'], left_on=['TELEFONE'], how="left")
# bases_teles_tels['BLACKLIST'] = bases_teles_tels['BLACKLIST'].astype('int32')
bases_teles_tels

In [ ]:
bloqueados_net = bases_teles_tels.query("BASE == 'GEVENUE NET' and BLACKLIST.notnull()")[['BASE','CONTRATO','TELEFONE','BLACKLIST']]

In [ ]:
bloqueados_net.query("CONTRATO == '002001000570'")

In [ ]:
bloqueados_net.to_excel(r"R:\TI\TELEFONIA\Mailings Prontos\2024\Agosto\02\Jorgean\BLIST_NET/TELS_NET_BLIST.xlsx",index=False)

In [ ]:
gp_teles = bases_teles_tels.groupby(['BASE','Faixa_de_Atraso']).agg({'TELEFONE':'count',
                                                         'BLACKLIST':'sum'}).reset_index()

gp_teles

In [ ]:
gp_teles['%%BLACKLIST'] = gp_teles['BLACKLIST']/gp_teles['TELEFONE'] * 100
gp_teles